# Implementation of the exciton coupling model

## Example

### Import modules

In [ ]:
import veloxchem as vlx
import numpy as np
import py3Dmol as p3d

### Set up molecule and basis set

In [ ]:
# molecule and basis

mol_xyz = """12
c2h4-dimer
C         -1.37731        1.01769       -0.71611
C         -0.04211        1.07142       -0.72602
H         -1.96225        1.74636       -0.16458
H         -1.90859        0.23094       -1.24174
H          0.49049        1.84498       -0.18262
H          0.54315        0.32947       -1.25941
C         -1.18880       -1.26220        2.03150
C          0.05470       -0.84410        2.28420
H         -1.37000       -2.18130        1.48410
H          0.91550       -1.41310        1.94850
H         -2.04870       -0.68100        2.34880
H          0.23460        0.08670        2.81250
"""
molecule = vlx.Molecule.from_xyz_string(mol_xyz)
basis = vlx.MolecularBasis.read(molecule, 'def2-svp')

In [ ]:
viewer = p3d.view(viewergrid=(1,1),width=300,height=200)
viewer.addModel(mol_xyz, 'xyz', viewer=(0,0))
viewer.setStyle({'stick': {}})
viewer.rotate(90,'x')
viewer.show()

### Set up the exciton model driver

In [ ]:
# exciton model setup

exmod_settings = {
    'fragments': '2',
    'atoms_per_fragment': '6',
    'charges': '0',
    'nstates': '5',
    'ct_nocc': '1',
    'ct_nvir': '1',
}

method_settings = {'dft': 'no'}

exmod_drv = vlx.ExcitonModelDriver()
exmod_drv.update_settings(exmod_settings, method_settings)

### Initializes exciton model Hamiltonian and transition dipoles

In [ ]:
monomer_natoms = list(exmod_drv.natoms)
n_monomers = len(monomer_natoms)
monomer_start_indices = [sum(exmod_drv.natoms[:i]) for i in range(n_monomers)]

npairs = n_monomers * (n_monomers - 1) // 2
total_LE_states = n_monomers * exmod_drv.nstates
total_CT_states = npairs * exmod_drv.ct_nocc * exmod_drv.ct_nvir * 2
total_num_states = total_LE_states + total_CT_states

exmod_drv.H = np.zeros((total_num_states, total_num_states))
exmod_drv.elec_trans_dipoles = np.zeros((total_num_states, 3))
exmod_drv.velo_trans_dipoles = np.zeros((total_num_states, 3))
exmod_drv.magn_trans_dipoles = np.zeros((total_num_states, 3))
exmod_drv.center_of_mass = molecule.center_of_mass()

exmod_drv.state_info = [
    {'type': '', 'frag': '', 'name': ''}
    for s in range(total_num_states)
]

excitation_ids = exmod_drv.get_excitation_ids()

### Run monomer calculations

In [ ]:
# monomer calculations

monomers_info = [{} for ind in range(n_monomers)]

for ind in range(n_monomers):
    monomer = molecule.get_sub_molecule(monomer_start_indices[ind],
                                        monomer_natoms[ind])
    monomer.set_charge(exmod_drv.charges[ind])
    monomer.check_multiplicity()

    scf_tensors = exmod_drv.monomer_scf(method_settings, ind, monomer, basis)
    tda_results = exmod_drv.monomer_tda(method_settings, ind, monomer, basis,
                                        scf_tensors)

    monomers_info[ind]['mo'] = scf_tensors['C_alpha']
    monomers_info[ind]['exc_energies'] = tda_results['exc_energies']
    monomers_info[ind]['exc_vectors'] = tda_results['exc_vectors']

    one_elec_ints = exmod_drv.get_one_elec_integrals(monomer, basis)
    trans_dipoles = exmod_drv.get_LE_trans_dipoles(monomer, basis,
                                                   one_elec_ints, scf_tensors,
                                                   tda_results)

    # LE states
    for s in range(exmod_drv.nstates):
        h = excitation_ids[ind, ind] + s
        # LE energies
        exmod_drv.H[h, h] = monomers_info[ind]['exc_energies'][s]
        # LE transition dipoles
        exmod_drv.elec_trans_dipoles[h, :] = trans_dipoles['electric'][s]
        exmod_drv.velo_trans_dipoles[h, :] = trans_dipoles['velocity'][s]
        exmod_drv.magn_trans_dipoles[h, :] = trans_dipoles['magnetic'][s]

### Run dimer calculations

In [ ]:
# dimer calculations

for ind_A in range(n_monomers):
    monomer_A = molecule.get_sub_molecule(monomer_start_indices[ind_A],
                                          monomer_natoms[ind_A])
    monomer_A.set_charge(exmod_drv.charges[ind_A])
    monomer_A.check_multiplicity()

    for ind_B in range(ind_A + 1, n_monomers):
        monomer_B = molecule.get_sub_molecule(monomer_start_indices[ind_B],
                                              monomer_natoms[ind_B])
        monomer_B.set_charge(exmod_drv.charges[ind_B])
        monomer_B.check_multiplicity()

        dimer = vlx.Molecule(monomer_A, monomer_B)
        dimer.check_multiplicity()

        mo_A = monomers_info[ind_A]['mo']
        mo_B = monomers_info[ind_B]['mo']

        nocc_A = monomer_A.number_of_alpha_electrons()
        nocc_B = monomer_B.number_of_alpha_electrons()
        nvir_A = mo_A.shape[1] - nocc_A
        nvir_B = mo_B.shape[1] - nocc_B

        nocc = nocc_A + nocc_B
        nvir = nvir_A + nvir_B

        mo = exmod_drv.dimer_mo_coefficients(monomer_A, monomer_B, basis, mo_A,
                                             mo_B)

        dimer_prop = exmod_drv.dimer_properties(dimer, basis, mo)

        dimer_energy = dimer_prop['energy']

        exc_vectors_A = monomers_info[ind_A]['exc_vectors']
        exc_vectors_B = monomers_info[ind_B]['exc_vectors']

        exc_vectors = []

        exc_vectors += exmod_drv.dimer_excitation_vectors_LE_A(
            exc_vectors_A, ind_A, nocc_A, nvir_A, nocc, nvir, excitation_ids)

        exc_vectors += exmod_drv.dimer_excitation_vectors_LE_B(
            exc_vectors_B, ind_B, nocc_A, nvir_A, nocc, nvir, excitation_ids)

        exc_vectors += exmod_drv.dimer_excitation_vectors_CT_AB(
            ind_A, ind_B, nocc_A, nvir_A, nocc, nvir, excitation_ids)

        exc_vectors += exmod_drv.dimer_excitation_vectors_CT_BA(
            ind_A, ind_B, nocc_A, nvir_A, nocc, nvir, excitation_ids)
        
        for c_vec in exc_vectors:
            exmod_drv.state_info[c_vec['index']]['frag'] = c_vec['frag']
            exmod_drv.state_info[c_vec['index']]['type'] = c_vec['type']
            exmod_drv.state_info[c_vec['index']]['name'] = c_vec['name']

        sigma_vectors = exmod_drv.dimer_sigma_vectors(dimer, basis, dimer_prop,
                                                      mo, exc_vectors)

        one_elec_ints = exmod_drv.get_one_elec_integrals(dimer, basis)
        trans_dipoles = exmod_drv.get_CT_trans_dipoles(
            dimer, basis, one_elec_ints, mo,
            exc_vectors[exmod_drv.nstates * 2:])

        # CT states
        for i_vec, (c_vec, s_vec) in enumerate(
                zip(exc_vectors[exmod_drv.nstates * 2:],
                    sigma_vectors[exmod_drv.nstates * 2:])):
            # CT energies
            energy = np.vdot(c_vec['vec'], s_vec['vec'])
            exmod_drv.H[c_vec['index'], c_vec['index']] = energy
            # CT transition dipoles
            exmod_drv.elec_trans_dipoles[
                c_vec['index'], :] = trans_dipoles['electric'][i_vec]
            exmod_drv.velo_trans_dipoles[
                c_vec['index'], :] = trans_dipoles['velocity'][i_vec]
            exmod_drv.magn_trans_dipoles[
                c_vec['index'], :] = trans_dipoles['magnetic'][i_vec]

        # LE(A)-LE(B) couplings
        for c_vec in exc_vectors[:exmod_drv.nstates]:
            for s_vec in sigma_vectors[exmod_drv.nstates:exmod_drv.nstates * 2]:
                coupling = np.vdot(c_vec['vec'], s_vec['vec'])
                exmod_drv.H[c_vec['index'], s_vec['index']] = coupling
                exmod_drv.H[s_vec['index'], c_vec['index']] = coupling

        # LE-CT couplings
        for c_vec in exc_vectors[:exmod_drv.nstates * 2]:
            for s_vec in sigma_vectors[exmod_drv.nstates * 2:]:
                coupling = np.vdot(c_vec['vec'], s_vec['vec'])
                exmod_drv.H[c_vec['index'], s_vec['index']] = coupling
                exmod_drv.H[s_vec['index'], c_vec['index']] = coupling

        # CT-CT couplings
        for c_vec in exc_vectors[exmod_drv.nstates * 2:]:
            for s_vec in sigma_vectors[exmod_drv.nstates * 2:]:
                if c_vec['index'] >= s_vec['index']:
                    continue
                coupling = np.vdot(c_vec['vec'], s_vec['vec'])
                exmod_drv.H[c_vec['index'], s_vec['index']] = coupling
                exmod_drv.H[s_vec['index'], c_vec['index']] = coupling

### Print the exciton model Hamiltonian

In [ ]:
np.set_printoptions(precision=6, suppress=True)

n = exmod_drv.nstates
H = exmod_drv.H

print('The LE(A)-LE(A) block:\n')
print(H[:n, :n], '\n')

print('The LE(A)-LE(B) block:\n')
print(H[:n, n:n*2], '\n')

print('The LE(A)-CT block:\n')
print(H[:n, n*2:], '\n')

print('The LE(B)-LE(B) block:\n')
print(H[n:n*2, n:n*2], '\n')

print('The LE(B)-CT block:\n')
print(H[n:n*2, n*2:], '\n')

print('The CT-CT block:\n')
print(H[n*2:, n*2:], '\n')

In [ ]:
for s, info in enumerate(exmod_drv.state_info):
    print(f'diabatic state {s+1:<5d}',
          info['type'] + '(' + info['frag'] + ')  ', info['name'])

### Get excitation energies and transition dipoles

In [ ]:
# Exciton model energies

eigvals, eigvecs = np.linalg.eigh(exmod_drv.H)

elec_trans_dipoles = np.matmul(eigvecs.T, exmod_drv.elec_trans_dipoles)
velo_trans_dipoles = np.matmul(eigvecs.T, exmod_drv.velo_trans_dipoles)
magn_trans_dipoles = np.matmul(eigvecs.T, exmod_drv.magn_trans_dipoles)

excitation_energies = []
oscillator_strengths = []
rotatory_strengths = []

for s in range(total_num_states):
    ene = eigvals[s]
    dip_strength = np.sum(elec_trans_dipoles[s, :]**2)
    f = (2.0 / 3.0) * dip_strength * ene

    velo_trans_dipoles[s, :] /= -ene
    magn_trans_dipoles[s, :] *= -0.5
    R = (-1.0) * np.vdot(velo_trans_dipoles[s, :], magn_trans_dipoles[s, :])
    
    excitation_energies.append(ene)
    oscillator_strengths.append(f)
    rotatory_strengths.append(R)

    print(f'S{s+1:<2d}  e={ene:<.8f}  f={f:<.4f}  R={R:<.4f}')

### Plot absorption and ECD spectra

In [ ]:
import matplotlib.pyplot as plt

def lorentzian(x, y, xmin, xmax, xstep, gamma):
    '''
    Lorentzian broadening function
    
    Call: xi,yi = lorentzian(energies, intensities, start energy, end energy, energy step, gamma)
    '''
    xi = np.arange(xmin,xmax,xstep); yi=np.zeros(len(xi))
    for i in range(len(xi)):
        for k in range(len(x)): yi[i] = yi[i] + y[k] * (gamma/2.) / ( (xi[i]-x[k])**2 + (gamma/2.)**2 )
    return xi,yi

plt.figure(figsize=(6,4))

x = np.array(excitation_energies) * 27.211385
y_abs = np.array(oscillator_strengths)
y_ecd = np.array(rotatory_strengths)

x0,y0 = lorentzian(x, y_abs, min(x)-1.0, max(x)+1.0, 0.01, 0.3)
plt.plot(x0,y0)
plt.show()

x1,y1 = lorentzian(x, y_ecd, min(x)-1.0, max(x)+1.0, 0.01, 0.3)
plt.plot(x1,y1)
plt.show()

In [ ]:
x_ref = 27.211385 * np.array([
    0.28006382, 0.31962256, 0.32767727, 0.33668306, 0.33909998, 0.34304918,
    0.35084393, 0.35401497, 0.36903968, 0.37361187, 0.38011701, 0.38276526,
    0.40267122, 0.40624580, 0.43639899, 0.44717735, 0.44833898, 0.45255633,
    0.45550674, 0.46005653, 0.46849404, 0.46969677, 0.47570344, 0.48304819,
    0.48700812, 0.49100297, 0.49327489, 0.49964034, 0.49968328, 0.50130547,
    0.53776443, 0.53813345, 0.54546741, 0.54863628, 0.55031253, 0.55090513,
    0.55234761, 0.55930002, 0.56037421, 0.56209463, 0.57460487, 0.57523583,
    0.57677425, 0.58646026, 0.58753148, 0.59186581, 0.60060163, 0.61066378,
    0.61104658, 0.61446650
])

y_abs_ref = np.array([
    0.0240, 1.1495, 0.0214, 0.0044, 0.0001, 0.0000, 0.0002, 0.0000, 0.0011,
    0.0000, 0.0066, 0.0006, 0.0004, 0.0054, 0.0001, 0.0001, 0.0004, 0.0000,
    0.0001, 0.0074, 0.0000, 0.0000, 0.0025, 0.0312, 0.0344, 0.0404, 0.0169,
    0.8469, 0.8348, 0.6263, 0.1069, 0.1388, 0.0152, 0.0016, 0.0006, 0.0044,
    0.1836, 0.0004, 0.0003, 0.0004, 0.0251, 0.1734, 0.4483, 0.0010, 0.0179,
    0.0003, 0.2370, 0.0482, 0.0327, 0.0045
])

y_ecd_ref = np.array([
    0.121385, -0.169583, 0.001083, 0.031189, 0.000045, 0.002472, 0.010582,
    0.001767, -0.025336, 0.004227, -0.013767, 0.017773, 0.001342, 0.003217,
    0.000049, -0.000133, 0.000727, 0.000614, 0.004981, -0.000802, -0.000001,
    -0.000003, 0.002611, 0.071303, -0.020126, 0.516321, -0.266109, -0.389929,
    -0.468289, 0.548157, 0.058227, -0.047284, 0.000152, 0.000046, 0.000452,
    0.033710, -0.067830, 0.006311, -0.000511, -0.000381, 0.263699, -0.017538,
    -0.233172, 0.000784, 0.028919, -0.000594, -0.004775, -0.017479, 0.000690,
    -0.028702
])

x0_ref,y0_ref = lorentzian(x_ref, y_abs_ref, min(x)-1.0, max(x)+1.0, 0.01, 0.3)
plt.plot(x0,y0,label='Exciton model')
plt.plot(x0_ref,y0_ref,label='TDDFT-TDA')
plt.legend()
plt.show()

x1_ref,y1_ref = lorentzian(x_ref, y_ecd_ref, min(x)-1.0, max(x)+1.0, 0.01, 0.3)
plt.plot(x1,y1,label='Exciton model')
plt.plot(x1_ref,y1_ref,label='TDDFT-TDA')
plt.legend()
plt.show()